test notebook for ShardVCFWf workflow
quickly thrown together workflow using WDL
takes a set of vcfs and shards by a set intervals (assumes bed format)
I wouldn't normally run this from localhost mac notebook, but here have modified so hopefully it will


    ##for this test purpose I'm using ldetect approx independent LD blocks form 1KG phase 1Europeans
    ##need to liftover from hg19 to hg38
    awk '$2 != "start"' /labshare/raph/datasets/ldetect/nygcresearch-ldetect-data-ac125e47bf7f/EUR/fourier_ls-all.bed \
    > /labshare/raph/datasets/ldetect/EUR.fourier_ls-all.hg19.bed

    INBED=/labshare/raph/datasets/ldetect/EUR.fourier_ls-all.hg19.bed
    CHAINFILE=/labseq/Genomes/Liftover_Chain_Files/hg19ToHg38.over.chain
    OUTBED=/labshare/raph/datasets/ldetect/eur.fourier_ls-all.hg38.bed
    UNMAPPEDBED=/labshare/raph/datasets/ldetect/eur.fourier_ls-all.hg38.unmapped.bed

    /labseq/tools/ucsc/liftOver ${INBED} ${CHAINFILE} ${OUTBED} ${UNMAPPEDBED}

    gsutil cp ${OUTBED} gs://nihnialng-ppmi-wgs/test/

    gsutil ls -lh gs://nihnialng-ppmi-wgs/test/


pull down intervals file

In [ ]:
%%bash
mkdir ~/data/ldetect
gsutil cp gs://nihnialng-ppmi-wgs/test/eur.fourier_ls-all.hg38.bed ~/data/ldetect/

split intervals bed file by the chromosomes present in the file

In [10]:
import pandas as pd

bedpath = "~/data/ldetect/eur.fourier_ls-all.hg38.bed"
outprefix = "~/data/ldetect/eur.fourier_ls-all.hg38"

df = pd.read_table(bedpath,sep='\t',header=None)
df.columns = ['chr','start','stop']
#save genome bed as individual chromosome beds
chromosomes = df["chr"].unique().tolist()
for thisChr in chromosomes:
    outfile = outprefix+"."+thisChr+".bed"
    df[df["chr"] == thisChr].to_csv(outfile,sep="\t",index=False,header=False)
    outputFiles.append(outfile)
    

In [ ]:
!ls -lh ~/data/ldetect/

In [ ]:
!head ~/data/ldetect/eur.fourier_ls-all.hg38.chr9.bed

In [ ]:
%%bash
#need to push per chr intervals to GCS
#-m is threaded -q is quiet
gsutil -mq cp ~/data/ldetect/eur.fourier_ls-all.hg38.chr*.bed gs://nihnialng-ppmi-wgs/test/intervals/
    
gsutil ls -lh gs://nihnialng-ppmi-wgs/test/intervals/

so everything above this point was just setting up interval files for the jobs not actually running the jobs

In [54]:
#setup local space for input json files
!mkdir -p ~/data/shardvcfs/jsons

In [97]:
#format the WF input jsons
import json

json_template = "/Users/gibbsr/code/scripts/ShardVCFWf.inputs.json"

interval_format = 'gs://nihnialng-ppmi-wgs/test/intervals/eur.fourier_ls-all.hg38.chr{thischr}.bed'
vcf_format = 'gs://nihnialng-ppmi-wgs/hg38/genotypes/ppmi.july2018.chr{thischr}.vqsr.vcf.gz'
vcf_index_format = 'gs://nihnialng-ppmi-wgs/hg38/genotypes/ppmi.july2018.chr{thischr}.vqsr.vcf.gz.tbi'
basename = 'ppmi.july2018'

chromosomes = list(map(str,range(1,23)))
chromosomes.extend(['X','Y'])

with open(json_template) as json_file:  
    data = json.load(json_file)
    data["ShardVCFWf.basename_prefix"] = basename
    
    for chromosome in chromosomes:
        data["ShardVCFWf.intervals_list_bed"] = interval_format.format(thischr=chromosome)
        data["ShardVCFWf.input_vcf"] = vcf_format.format(thischr=chromosome)
        data["ShardVCFWf.input_vcf_index"] = vcf_index_format.format(thischr=chromosome)
        outfile = "/Users/gibbsr/data/shardvcfs/jsons/chr"+chromosome+".json"
        with open(outfile,'w') as outfile:
            json.dump(data,outfile,sort_keys=True,indent=4)

In [149]:
%%bash
#for PPMI the autosome vcf have vqsr in name but X and Y have raw, modify X and Y json
#sed on linux good, sed on mac lame
sed -i.bak s"/vqsr/raw/"g ~/data/shardvcfs/jsons/chrX.json
sed -i.bak s"/vqsr/raw/"g ~/data/shardvcfs/jsons/chrY.json

rm ~/data/shardvcfs/jsons/chrX.json.bak
rm ~/data/shardvcfs/jsons/chrY.json.bak

In [ ]:
%%bash
#pull down workflow file and generic options
gsutil -mq cp gs://nihnialng-ppmi-wgs/test/ShardVCFWf.wdl data/shardvcfs/
gsutil -mq cp gs://nihnialng-ppmi-wgs/test/generic.google-papi.options.json data/shardvcfs/

In [ ]:
%%bash
#set some variables
PROJECT_ID=pd-genome
COHORT=ppmi
COHORT_BUCKET=gs://nihnialng-ppmi-wgs
PIPEYAML=gs://nihnialng-pd-wgs/tools/wdl_pipeline.yaml
WFWDL=~/data/shardvcfs/ShardVCFWf.wdl
WFOPTIONS=~/data/shardvcfs/generic.google-papi.options.json
LCCOHORT=$(echo ${COHORT} | awk '{ print tolower($1) }')

#execute the workflow (wf) on the cloud
IN_CHROMOSOMES="1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,X,Y"
CHRS=$(echo ${IN_CHROMOSOMES//,/ })
for CHR in ${CHRS[@]}
do
LCCHR=$(echo ${CHR} | awk '{ print tolower($1) }')
gcloud alpha genomics pipelines run \
--project ${PROJECT_ID} \
--pipeline-file ${PIPEYAML} \
--zones us-central1-f \
--memory 14 \
--logging ${COHORT_BUCKET}/test/logs/shardvcf \
--inputs-from-file WDL=${WFWDL} \
--inputs-from-file WORKFLOW_INPUTS=~/data/shardvcfs/jsons/chr${CHR}.json \
--inputs-from-file WORKFLOW_OPTIONS=${WFOPTIONS} \
--inputs WORKSPACE=${COHORT_BUCKET}/test/workspace \
--inputs OUTPUTS=${COHORT_BUCKET}/test/chr${CHR}shards/ \
--labels=pipe=shardvcf,cohort=${LCCOHORT},chrom=${LCCHR}
done


In [ ]:
%%bash
#see if there are instances running the job
PROJECT_ID=pd-genome
COHORT=ppmi

echo "full job worker node count"
gcloud compute instances list --project ${PROJECT_ID} | grep RUNNING | wc -l
echo "job managers"
gcloud compute instances list --project ${PROJECT_ID} --filter "labels.pipe=shardvcf labels.cohort=${COHORT}"

In [ ]:
%%bash
#grab the output operations ID from above and check if running
OPID=EMHCoOzqLBiM99-2ze7__AkgpIDW36QGKg9wcm9kdWN0aW9uUXVldWU

gcloud alpha genomics operations describe ${OPID} \
--format='yaml(done, error, metadata.events)'

In [195]:
#check that your files are there
!gsutil ls -lh gs://nihnialng-ppmi-wgs/test/chr*shards/*.vcf.gz | wc -l

    1250


In [196]:
#GGP WDL runner requires WF final output dir to be empty, since I iterated over chromosomes launching WF per chr \
#that means I output each to its own bucket path, so merge them into single bucket path
!gsutil -mq mv gs://nihnialng-ppmi-wgs/test/chr*shards/*.vcf.gz gs://nihnialng-ppmi-wgs/test/shards/

In [227]:
#see if all the files are there
!gsutil ls -lh gs://nihnialng-ppmi-wgs/test/shards/*.vcf.gz | wc -l

    1626


check that all the expected files are there

In [228]:
%%bash
gsutil ls gs://nihnialng-ppmi-wgs/test/shards/*.vcf.gz > ~/data/shardvcfs/found.files.txt
sed -i.bak s"/gs:\/\/nihnialng-ppmi-wgs\/test\/shards\///"g ~/data/shardvcfs/found.files.txt
sed -i.bak s"/ppmi.july2018_//"g ~/data/shardvcfs/found.files.txt
sed -i.bak s"/.vcf.gz//"g ~/data/shardvcfs/found.files.txt

In [229]:
!head ~/data/shardvcfs/found.files.txt

chr10_101189482_102620653
chr10_10207433_12544798
chr10_102620653_104935290
chr10_104935290_106966928
chr10_106966928_108557947
chr10_108557947_110801735
chr10_110801735_113568673
chr10_113568673_114661647
chr10_114661647_117764423
chr10_117764423_118831841


In [230]:
expectedFile = "~/data/ldetect/eur.fourier_ls-all.hg38.bed"
foundFile = "~/data/shardvcfs/found.files.txt"
outFile = "~/data/shardvcfs/missing.hg38.bed"

expected = pd.read_table(expectedFile,sep='\t',header=None)
expected.columns = ['chr','start','stop']
found = pd.read_table(foundFile,sep='\t',header=None)
found.columns = ['interval']

expected['interval'] = expected["chr"].astype(str)+"_"+expected["start"].astype(str)+"_"+expected["stop"].astype(str)

missing = expected[~expected.interval.isin(found.interval)]
missing.shape
missing.head()

missingOut = missing[['chr','start','stop']]
print(missingOut.shape)
print(missingOut.head())
missingOut.to_csv(outFile,sep="\t",index=False,header=False)



(0, 3)
Empty DataFrame
Columns: [chr, start, stop]
Index: []


some still failed and are missing, hey it is complex broad stuff
so could use new missing bed as input to shard wf or also just run with bash loop over a bcftools yaml
both are not great clunky, keep repeating on missing til done, but that's how it is with a lot of these pipe lines
this also gives examples of running yaml
also looks like some of the cromwell instances were just doing nothing after a bit 
but don't have time to debug and tracing thru the cromwell is really unpleasant

In [ ]:
%%bash
#pull down the yaml
#sometimes juypter bash won't execute gsutil commands correctly so you may have to run in terminal
gsutil -m cp gs://nihnialng-ppmi-wgs/test/RegionSubsetVCF.yaml data/shardvcfs/

In [223]:
%%bash

COHORT=ppmi
COHORTBUILD=${COHORT}.july2018
PROJECT_ID=pd-genome
PIPEYAML=~/data/shardvcfs/RegionSubsetVCF.yaml
COHORT_BUCKET=gs://nihnialng-ppmi-wgs
INTERVALFILE=~/data/shardvcfs/missing.hg38.bed

LCCOHORT=$(echo ${COHORT} | awk '{ print tolower($1) }')

while read -r INTERVAL
do
CHROMOSOME=$(echo ${INTERVAL} | awk '{print $1}')
STARTBP=$(echo ${INTERVAL} | awk '{print $2}')
STOPBP=$(echo ${INTERVAL} | awk '{print $3}')
FRMTINTERVAL=$(echo "${CHROMOSOME}_${STARTBP}_${STOPBP}")
LCINTERVAL=$(echo ${FRMTINTERVAL} | awk '{ print tolower($1) }')
echo -n "${COHORT} ${INTERVAL} OPID="

gcloud alpha genomics pipelines run \
--project ${PROJECT_ID} \
--pipeline-file ${PIPEYAML} \
--preemptible \
--logging ${COHORT_BUCKET}/test/logs/regionsubsetvcf/ \
--inputs INPUTVCF=${COHORT_BUCKET}/hg38/genotypes/${COHORTBUILD}.${CHROMOSOME}.vqsr.vcf.gz \
--inputs INPUTVCFINDEX=${COHORT_BUCKET}/hg38/genotypes/${COHORTBUILD}.${CHROMOSOME}.vqsr.vcf.gz.tbi \
--inputs CHROMOSOME=${CHROMOSOME} \
--inputs STARTBP=${STARTBP} \
--inputs STOPBP=${STOPBP} \
--outputs OUTVCF=${COHORT_BUCKET}/test/shards/${COHORTBUILD}_${FRMTINTERVAL}.vcf.gz \
--outputs OUTVCFINDEX=${COHORT_BUCKET}/test/shards/${COHORTBUILD}_${FRMTINTERVAL}.vcf.gz.tbi \
--labels=pipe=regionsubsetvcf,cohort=${LCCOHORT},interval=${LCINTERVAL}
done < ${INTERVALFILE}

ppmi chr1	109761309	111199115 OPID=ppmi chr2	180448012	181401304 OPID=ppmi chr2	235134906	236540389 OPID=ppmi chr4	124289707	125559731 OPID=ppmi chr4	187551827	188772357 OPID=

Running [operations/EJye34frLBiekeW7p5-6vXcgpIDW36QGKg9wcm9kdWN0aW9uUXVldWU].
Running [operations/EKyk34frLBic1M-coIDSsEkgpIDW36QGKg9wcm9kdWN0aW9uUXVldWU].
Running [operations/EP6q34frLBiGmcm31u_P_2IgpIDW36QGKg9wcm9kdWN0aW9uUXVldWU].
Running [operations/EM6x34frLBiugsfv6-jVlB8gpIDW36QGKg9wcm9kdWN0aW9uUXVldWU].
Running [operations/EPW334frLBjrztay1enoueMBIKSA1t-kBioPcHJvZHVjdGlvblF1ZXVl].


In [226]:
%%bash
#see if there are instances running the job
PROJECT_ID=pd-genome
COHORT=ppmi

echo "full job worker node count"
gcloud compute instances list --project ${PROJECT_ID} --filter "labels.pipe=regionsubsetvcf labels.cohort=${COHORT}" | grep RUNNING | wc -l

full job worker node count
       0


now go back up and recheck that all files are present and if not find missing, may want to run next pass of yaml without the "--preemptible \" portion

cell to go back up to
    #see if all the files are there
    !gsutil ls -lh gs://nihnialng-ppmi-wgs/test/shards/*.vcf.gz | wc -l

%%bash
#if everything is done and looks right, clean up

##be careful using wildcards, recursion and fat fingers on deletes
#delete can be a little slow sometimes
gsutil -mq rm -r gs://nihnialng-ppmi-wgs/test/logs/shardvcf
gsutil -mq rm -r gs://nihnialng-ppmi-wgs/workspace/ShardVCFWf
gsutil -mq rm -r gs://nihnialng-ppmi-wgs/test/chr*shards

In [232]:
!gsutil -mq rm -r gs://nihnialng-ppmi-wgs/test/logs/shardvcf

CommandException: 1 files/objects could not be removed.


In [233]:
!gsutil -mq rm -r gs://nihnialng-ppmi-wgs/test/logs/regionsubsetvcf

In [ ]:
!gsutil -mq rm -r gs://nihnialng-ppmi-wgs/test/workspace/ShardVCFWf